# 统计第一次停留点和只访问过一次停留点的占比

按照 Understanding predictability and exploration in human mobility 文中的定义来计算。

## 1 新地点（Exploration）

对 **每个用户**，按时间排序的地点序列：

- 若当前位置 **在该用户历史中从未出现过**
- 则该一次访问记为 **exploration**

例子（论文原文）：
```text
A B A C B C  
→ 1 1 0 1 0 0
```

统计量：

$$
\text{Exploration Ratio}
= \frac{\#\text{exploration visits}}{\#\text{total visits}}
$$

---

## 2. 只访问过一次的地点

对 **每个用户**：

- 统计其所有访问过的唯一地点
- 其中 **访问次数 = 1 的地点数**
- 占 **唯一地点总数** 的比例

$$
\text{Once-Visited Location Ratio}
= \frac{\#\text{locations visited once}}{\#\text{unique locations}}
$$


In [ ]:
import pandas as pd

# 1. 读取数据
df = pd.read_csv(
    "./Data/Output/all_users_context_combined.csv",
    parse_dates=["stime"]
)

# 2. 按 user + time 排序（论文要求是序列）
df = df.sort_values(["userID", "stime"])

def compute_user_stats(user_df):
    """
    对单个用户计算：
    1) exploration 占比
    2) 只访问一次的地点占比
    """
    locations = user_df["grid"].tolist()

    # ---------- Exploration ----------
    seen = set()
    exploration_flags = []

    for loc in locations:
        if loc in seen:
            exploration_flags.append(0)
        else:
            exploration_flags.append(1)
            seen.add(loc)

    exploration_ratio = sum(exploration_flags) / len(exploration_flags)

    # ---------- Visited only once ----------
    loc_counts = pd.Series(locations).value_counts()
    once_visited_ratio = (loc_counts == 1).sum() / len(loc_counts)

    return pd.Series({
        "exploration_ratio": exploration_ratio,
        "once_visited_location_ratio": once_visited_ratio,
        "num_visits": len(locations),
        "num_unique_locations": len(loc_counts)
    })

# 3. 按用户统计
user_stats = (
    df.groupby("userID", group_keys=False)
      .apply(compute_user_stats)
      .reset_index()
)



C:\Users\Administrator\AppData\Local\Temp\ipykernel_5840\1574218667.py:47: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_user_stats)


In [ ]:
print(user_stats.head())
user_stats.to_csv('./Data/Output/GeoLife_ExplorationRatio_OnceVisitedLocationRatio.csv')

In [ ]:
import pandas as pd

# 1. 读取数据
df = pd.read_csv(
    "./Data/MoreUser/all.csv",
    parse_dates=["stime"]
)

# 2. 按 user + time 排序（论文要求是序列）
df = df.sort_values(["userID", "stime"])

def compute_user_stats(user_df):
    """
    对单个用户计算：
    1) exploration 占比
    2) 只访问一次的地点占比
    """
    locations = user_df["grid"].tolist()

    # ---------- Exploration ----------
    seen = set()
    exploration_flags = []

    for loc in locations:
        if loc in seen:
            exploration_flags.append(0)
        else:
            exploration_flags.append(1)
            seen.add(loc)

    exploration_ratio = sum(exploration_flags) / len(exploration_flags)

    # ---------- Visited only once ----------
    loc_counts = pd.Series(locations).value_counts()
    once_visited_ratio = (loc_counts == 1).sum() / len(loc_counts)

    return pd.Series({
        "exploration_ratio": exploration_ratio,
        "once_visited_location_ratio": once_visited_ratio,
        "num_visits": len(locations),
        "num_unique_locations": len(loc_counts)
    })

# 3. 按用户统计
user_stats = (
    df.groupby("userID", group_keys=False)
      .apply(compute_user_stats)
      .reset_index()
)

print(user_stats.head())
user_stats.to_csv('./Data/Output/MoreUser_ExplorationRatio_OnceVisitedLocationRatio.csv')